# Descargando datos

## Ejercicio 1
Utiliza los archivos json descargados con el detalle a nivel estatal, y genera unos 3 DataFrames con información sobre personas desaparecidas dependiendo de diferentes características.

In [1]:
import os  # Para manejo de archivos y directorios
import urllib.request # Una forma estandard de descargar datos
# import requests # Otra forma no de las librerías de uso comun

import datetime # Fecha de descarga
import pandas as pd # Solo para ver el archivo descargado
import zipfile # Descompresión de archivos

In [3]:
# pwd
print(os.getcwd())

#  Estos son los datos que vamos a descargar y donde vamos a guardarlos
desaparecidos_RNPDNO_url = "http://www.datamx.io/dataset/fdd2ca20-ee70-4a31-9bdf-823f3c1307a2/resource/d352810c-a22e-4d72-bb3b-33c742c799dd/download/desaparecidos3ago.zip"
desaparecidos_RNPDNO_archivo = "desaparecidosRNPDNO.zip"
desaparecidos_corte_nacional_url = "http://www.datamx.io/dataset/fdd2ca20-ee70-4a31-9bdf-823f3c1307a2/resource/4865e244-cf59-4d39-b863-96ed7f45cc70/download/nacional.json"
desaparecidos_corte_nacional_archivo = "desaparecidos_nacional.json"
subdir = "./data/"

c:\Users\DELL\Documents\Areas\MCD\1S\Ingeniería de Características\ingesta de datos\Ingesta-de-datos-ejercicios


In [4]:
if not os.path.exists(desaparecidos_RNPDNO_archivo):
    if not os.path.exists(subdir):
        os.makedirs(subdir)
    urllib.request.urlretrieve(desaparecidos_RNPDNO_url, subdir + desaparecidos_RNPDNO_archivo)  
    with zipfile.ZipFile(subdir + desaparecidos_RNPDNO_archivo, "r") as zip_ref:
        zip_ref.extractall(subdir)
    
    urllib.request.urlretrieve(desaparecidos_corte_nacional_url, subdir + desaparecidos_corte_nacional_archivo)  

    with open(subdir + "info.txt", 'w') as f:
        f.write("Archivos sobre personas desaparecidas\n")
        info = """
        Datos de desaparecidos, corte nacional y desagregación a nivel estatal, 
        por edad, por sexo, por nacionalidad, por año de desaparición y por mes
        de desaparición para los últimos 12 meses.

        Los datos se obtuvieron del RNPDNO con fecha de 03 de agosto de 2021
        (la base de datos no se ha actualizado últimamente) 

        """ 
        f.write(info + '\n')
        f.write("Descargado el " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
        f.write("Desde: " + desaparecidos_RNPDNO_url + "\n")
        f.write("Nombre: " + desaparecidos_RNPDNO_archivo + "\n")
        f.write("Agregados nacionales descargados desde: " + desaparecidos_corte_nacional_url + "\n")
        f.write("Nombre: " + desaparecidos_corte_nacional_archivo + "\n")

In [ ]:
id_Sonora = 26


## Ejercicio 2
Entender la estructura del archivo xml de poetas, hacer un query de otro tema que consideren interesante y generar un DataFrame con la información más importante. No olvides de comentar tu código y explicar la estructura del archivo xml

In [4]:
%pip install requests

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)

   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   -------- ------------------------------- 1/5 [idna]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   ------------------------ --------------- 3/5 [certifi]
   -------------------------------- ------- 4/5 [

In [6]:
import requests

# URL de la API de MediaWiki de Wikipedia en español
API_URL = "https://es.wikipedia.org/w/api.php"

# Cabecera para identificar nuestra aplicación (buena práctica)
HEADERS = {
    'User-Agent': 'WikiXMLPseudoDump/1.0 (a225230115@unison.mx)'
}

def get_page_titles(category_title):
    """Obtiene la lista de títulos de páginas de una categoría."""
    titles = []
    cmcontinue = None 
    while True:
        params = {
            "action": "query",
            "format": "json",
            "list": "categorymembers",
            "cmtitle": category_title,
            "cmlimit": "500",  # El límite máximo por petición
            "cmcontinue": cmcontinue
        }     
        response = requests.get(API_URL, params=params, headers=HEADERS)
        data = response.json()
        
        for member in data['query']['categorymembers']:
            titles.append(member['title'])           
        if 'continue' in data:
            cmcontinue = data['continue']['cmcontinue']
        else:
            break           
    return titles

def get_page_content_in_xml(page_titles):
    """Obtiene el contenido de las páginas en formato XML."""
    params = {
        "action": "query",
        "format": "xml",
        "prop": "revisions",
        "rvprop": "content",
        "rvslots": "main",
        "titles": "|".join(page_titles)
    }
    response = requests.get(API_URL, params=params, headers=HEADERS)
    return response.content


In [ ]:
categoria = "Académicos e investigadores de la Universidad de Sonora"
#Otros interesantes son "Escritores de Sonora", "Alumnado de la Universidad de Sonora"
    
print(f"Obteniendo la lista de entradas de '{categoria}'...")
titles = get_page_titles('Categoría:'+categoria)

print(f"Se encontraron {len(titles)} entradas.")
if not titles:
  raise ValueError("No se encontraron páginas en la categoría especificada.")

batch_size = 50 # Lote de títulos para la segunda petición (máximo 50)
all_xml_data = []

for i in range(0, len(titles), batch_size):
  batch_titles = titles[i:i + batch_size]
  xml_content = get_page_content_in_xml(batch_titles)
  all_xml_data.append(xml_content)
  print(f"Procesadas entradas {i + 1} a {i + batch_size} de {len(titles)}")

Obteniendo la lista de entradas de 'Académicos e investigadores de la Universidad de Sonora'...
Se encontraron 16 entradas.
Procesadas entradas 1 a 50 de 16


In [ ]:
import xml.etree.ElementTree as et
# Combinar todos los XML en un solo archivo
root = et.Element(categoria.lower().replace(" ", "_"))
for xml_data in all_xml_data:
  # El contenido XML de la API tiene un elemento <api> y dentro <query>, que necesitamos para el contenido
  api_root = et.fromstring(xml_data)
  query_element = api_root.find('query')
  if query_element:
    # Los elementos <page> son los que contienen la información de cada poeta
    for page_element in query_element.findall('pages/page'):
      root.append(page_element)
tree = et.ElementTree(root)

# Guardamos en un archivo con el mismo nombre que la categoría (con .xml)
output_filename = categoria.lower().replace(" ", "_") + ".xml"
with open(output_filename, "wb") as f:
      # Escribe el XML completo al archivo
        tree.write(f, encoding='utf-8', xml_declaration=True)
        print(f"\nArchivo '{output_filename}' creado exitosamente con la información completa.")


Archivo 'académicos_e_investigadores_de_la_universidad_de_sonora.xml' creado exitosamente con la información completa.


C:\Users\DELL\AppData\Local\Temp\ipykernel_23352\1469790014.py:8: DeprecationWarning: Testing an element's truth value will always return True in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  if query_element:


In [27]:
import re
def extract_ficha_data(text):
    start = text.find("{{Ficha de ")
    if start == -1:
        return None

    i = start + 2  # skip starting {{
    brace_count = 2
    while i < len(text) and brace_count > 0:
        if text[i:i+2] == "{{":
            brace_count += 2
            i += 2
        elif text[i:i+2] == "}}":
            brace_count -= 2
            i += 2
        else:
            i += 1

    block = text[start:i]
    
    # Extract key-value pairs
    fields = re.findall(r'\| *([^=|]+)\s*=\s*(.*)', block)
    return fields

In [71]:
docentes = et.parse(output_filename)
nombres = []
escuelas = []
fechasNacimientos= []
for docente in docentes.getroot():
    #print(docente.attrib['title'])
    data = extract_ficha_data(docente[0][0][0][0].text) #para obtener el texto del articulo de wikipedia
    nombres.append(docente.attrib['title']) #nombre del articulo de wikipedia
    if data:
        #print(data)
        escuela_no_encontrada = True
        fecha_no_encontrada = True
        for key, value in data:
            if key.strip() == "alma máter": #para obtener dodne estudiaron
                escuela_no_encontrada = False
                escuelas.append(value.strip().replace("[[","").replace("]]",""))
                #print(f"    {key.strip()}: {value.strip().replace("[[","").replace("]]","")}")
            if key.strip() == "fecha de nacimiento":#para obtener cuando nacieron
                fecha_no_encontrada = False
                fechasNacimientos.append(value.strip().replace("[[","").replace("]]",""))
        if escuela_no_encontrada:
            escuelas.append(None)
        if fecha_no_encontrada:
            fechasNacimientos.append(None)
    else:
        escuelas.append(None) #manejar las datos faltantes
        fechasNacimientos.append(None)
[nombres, escuelas,fechasNacimientos]
#[len(nombres), len(escuelas), len(fechasNacimientos)] #verificar output

[['Alfonso López Corral',
  'Anna Ivette Rodríguez Navarro',
  'Daniel Avechuco Cabrera',
  'Elizabeth Araux Sánchez',
  'Elizabeth Cejudo Ramos',
  'Emma Susana Speratti Piñero',
  'Françoise Pérus',
  'Germán Viveros Maldonado',
  'Judith Tanori Córdova',
  'Martha Bracho',
  'Martha Guzmán Partida',
  'María Rita Plancarte Martínez',
  'Miguel Manríquez Durán',
  'Olivia Gutu Ocampo',
  'Tristana Landeros',
  'Zarina Estrada Fernández'],
 [None,
  'COLSON, Universidad de Sonora',
  'Universidad de Sonora',
  None,
  'Universidad Nacional Autónoma de México. Instituto de Investigaciones Históricas/Facultad de Filosofía y Letras',
  None,
  None,
  None,
  None,
  None,
  None,
  'Universidad de Sonora',
  None,
  None,
  None,
  None],
 ['1979',
  None,
  '{{fecha|1985}}',
  None,
  '27 de junio de 1980',
  None,
  None,
  None,
  None,
  '6 de febrero de 1927',
  None,
  None,
  '7 de septiembre de 1957',
  None,
  '1974',
  None]]

In [72]:
import pandas as pd
docentesUnison = pd.DataFrame(data={'nombre':nombres,"escuela":escuelas,"fecha_nacimiento":fechasNacimientos})
docentesUnison

,nombre,escuela,fecha_nacimiento
0,Alfonso López Corral,None,1979
1,Anna Ivette Rodríguez Navarro,"COLSON, Universidad de Sonora",None
2,Daniel Avechuco Cabrera,Universidad de Sonora,{{fecha|1985}}
3,Elizabeth Araux Sánchez,None,None
4,Elizabeth Cejudo Ramos,Universidad Nacional Autónoma de México. Insti...,27 de junio de 1980
5,Emma Susana Speratti Piñero,None,None
6,Françoise Pérus,None,None
7,Germán Viveros Maldonado,None,None
8,Judith Tanori Córdova,None,None
9,Martha Bracho,None,6 de febrero de 1927


# Usando la API para obtener datos sobre personas desaparecidas del RNPDNO

## Ejercicio 1
Probar con diferentes consultas y tratar de inferir los valores que pueden tomar (o buscarlas en la documentación de la API) las diferentes variables que pueden servir para encontrar búsquedas más específicas.

Por ejemplo, ¿Como podríamos consultar las estadísticas sobre mujeres desaparecidas en el municipio de Cajeme?

## Ejercicio 2
¿Como podemos sacar lo que pasa en todo el estado, por municipios y por colonias? Intentalo.

## Ejercicio 3
¿Se puede hacer por municipio? ¿En forma programática? ¿Para algun caso especial? Intentalo

## Ejercicio 4
Extrae alguna información del conjunto de tados que pienses que es relevante, y explica porqué.